In [1]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

In [2]:
import torch_geometric.utils
import torch

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [6]:
import lightning.pytorch as pl
import PPIGraph

# Test custom torch dataset

In [7]:
from torch_geometric.loader import DataLoader


In [8]:
condition_mapping = {'HCC827Ctrl': 0, 'HCC827Osim': 1}


In [9]:
graph_path = data_dir / 'OCT Cell Culture' / 'Whole' / 'graphs' 

dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)
train_set, val_set, test_set = PPIGraph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [10]:
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Dataset: GraphDataset(2059):
Number of graphs: 2059
Number of features: 5
Number of classes: 2


In [11]:
print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')

Train set: 989, test set: 823, val set: 247


In [12]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 12530], pos=[2409, 2], labels=[2409, 5], nuclei=[2409], weight=[12530], condition=[32], fov=[32], id=[32], train_mask=[2409], test_mask=[2409], x=[2409, 5], y=[32], edge_weight=[12530], name=[32], batch=[2409], ptr=[33])



# Test model

## Manual train GCN compoenents

In [13]:
def train(model, loader, epochs, criterion, optimizer, device, mlp=True):
    model.train()
    for epoch in range(epochs):
        for step, data in enumerate(loader):
            data = data.to(device)
            optimizer.zero_grad()
            if mlp:
                input_data = data.x.to(device)
                out = model(input_data)
            else:
                x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight
                out = model(x, edge_index, edge_weight=edge_weight)
            loss = criterion(out, data.nuclei)
            loss.backward()
            optimizer.step()
        if epoch%10==0:
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
            
def test(model, loader, device, mlp=True):
    model.eval()
    test_accs = [] 

    for step, data in enumerate(test_loader):
        data = data.to(device)
        if mlp:
            input_data = data.x.to(device)
            out = model(input_data)
        else:
            x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight
            out = model(x, edge_index, edge_weight=edge_weight)
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        test_correct = pred == data.nuclei  # Check against ground-truth labels.
        test_acc = int(test_correct.sum()) / len(pred)
        test_accs.append(test_acc)
        print(test_acc)
        
    return test_accs 

In [14]:
AVAIL_GPUS = [1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [15]:
# %time
# # Create model, criterion and optimizer
# model_gcn = PPIGraph.GNNModel("GCN", 5, 16, 2).to(device)
# criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
# optimizer = torch.optim.Adam(model_gcn.parameters(), lr=0.005, weight_decay=5e-4)

# # Train Loop
# train(model_gcn, train_loader, 10, criterion, optimizer, device, mlp=False)
# test_accs = test(model_gcn, test_loader, device, mlp=False)

## Lightning train

In [16]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [17]:
condition = 'base'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / "saved_models" / f"Graph_GNNs_{condition}" 
project_name = f'PLA_041623_{condition}'

In [18]:
AVAIL_GPUS = [1]
BATCH_SIZE = 64 if AVAIL_GPUS else 32

# Setting the seed
pl.seed_everything(42)

num_layers = [2,3,4]
hiddens = [16, 32, 64]
pools = ['mean', 'max', 'sum', 'attention', 'attention2']

epochs = 30

Global seed set to 42


In [19]:
# for NUM_LAYERS, HIDDEN_CHANNELS, pool in list(itertools.product(*[num_layers, hiddens, pools])):
    
#     # Path to the folder where the pretrained models are saved
#     CHECKPOINT_PATH = checkpoint_folder / f'{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot' / pool
#     CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

#     # Run training
#     models = ['GCN']
#     for model_name in models:
#         run = wandb.init(project=project_name, name=model_name+f'_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot')
#         model, result, trainer = PPIGraph.train_graph_classifier(model_name, 
#                                                                  train_set, 
#                                                                  val_set, 
#                                                                  test_set, 
#                                                                  dataset, 
#                                                                  CHECKPOINT_PATH, 
#                                                                  AVAIL_GPUS, 
#                                                                  in_channels=5,
#                                                                  hidden_channels=HIDDEN_CHANNELS, 
#                                                                  out_channels = HIDDEN_CHANNELS,
#                                                                  num_layers=NUM_LAYERS, epochs=epochs,
#                                                                  embedding=False,
#                                                                  graph_pooling=pool)
#         run.finish()

In [20]:
for NUM_LAYERS, HIDDEN_CHANNELS, pool in list(itertools.product(*[num_layers, hiddens, pools])):
    
    # Path to the folder where the pretrained models are saved
    CHECKPOINT_PATH = checkpoint_folder / f'{NUM_LAYERS}_{HIDDEN_CHANNELS}_emb' / pool
    CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

    # Run training
    models = ['GCN']
    for model_name in models:
        run = wandb.init(project=project_name, name=model_name+f'_{NUM_LAYERS}_{HIDDEN_CHANNELS}_emb')
        model, result, trainer = PPIGraph.train_graph_classifier(model_name, 
                                                                 train_set, 
                                                                 val_set, 
                                                                 test_set, 
                                                                 dataset, 
                                                                 CHECKPOINT_PATH, 
                                                                 AVAIL_GPUS, 
                                                                 in_channels=HIDDEN_CHANNELS,
                                                                 hidden_channels=HIDDEN_CHANNELS, 
                                                                 out_channels = HIDDEN_CHANNELS,
                                                                 num_layers=NUM_LAYERS, epochs=epochs,
                                                                 embedding=True,
                                                                 graph_pooling=pool)
        run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


Global seed set to 42
C:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Linear           | 96    
1 | model       | GNNModel         | 576   
2 | head        | Sequential       | 154   
3 | loss_module | CrossEntropyLoss | 0     
4 | metricf1    | BinaryF1Score    | 0     
5 | metricauc   | BinaryAUROC      | 0     
-------------------------------------------------
826       Trainable params
0     

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc_epoch,▁▅▆▆▇▆▅██▆▅▆▇▇▇▄▆▇▆▆▇▆▇▇▆▇▇▇▅▆
train_acc_step,▄▇▆▅▅▄▇▄▇▆▅▅▆▃▆▇▇▅▇▇▁█▅▅█▆▅▆▆█▆▅▅▆▇▅▆▇█▃
train_loss_epoch,█▅▃▃▁▃▄▁▁▃▂▁▂▂▂▃▂▃▃▃▂▂▂▃▂▂▂▁▃▃
train_loss_step,▇▃▄▄▄▃▃▇▂▃▇▃▃▅▂▂▂▄▂▂▇▂▅▄▂▃▃▃▃▁▅▄▃▃▂█▅▂▂▅
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,█▅▃▁▇▅▂▄▇▇█▇▄▁▆▅▄█▇▇█▄▆▆▆▄▆▂▅▅


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Linear           | 96    
1 | model       | GNNModel         | 576   
2 | head        | Sequential       | 154   
3 | loss_module | CrossEntropyLoss | 0     
4 | metricf1    | BinaryF1Score    | 0     
5 | metricauc   | BinaryAUROC      | 0     
-------------------------------------------------
826       Trainable params
0         Non-trainable params
826       Total params
0.003     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=30` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc_epoch,▂▁▄▄▄▄▃▅▇▂▅▆▄▆▅▇▇▅▄▆▆▄▆▇▆▇█▇▇▅
train_acc_step,▂▅▇▃▅▇▁▅▅▃▂▅▅▂▅▇▇▃▇▅▃█▆▃▅█▂▇▆█▅█▆▆█▆▆▇█▂
train_loss_epoch,█▄▃▄▃▄▃▂▃▃▂▂▃▂▃▂▂▃▃▁▂▂▂▂▂▁▂▂▁▂
train_loss_step,▃▃▂▃▃▂▄▃▃▃▄▃▃▄▃▃▃▃▂▃▅▂▃▃▃▂▄▂▂▂▃▂▃▂▂▂▃▂▁█
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▁▁▁▁▁▁▁▂▂█▆▃▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁


...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc_epoch,▁▁▁▃█▅▂▅▆▅▅▅▇▇█▆█▇▆▃▆█▇▅▆▃▅▇▆▆
train_acc_step,██▃▂▇▅▃▅▅▅▄▇▇▁▇▅▄▅█▅▄▇▅▅▅▇▅▄█▇█▇▇▇▅▄▇█▃▅
train_loss_epoch,█▂▂▂▁▂▃▂▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▂
train_loss_step,▁▂▆█▃▆▄▅▄▅▆▂▂▅▃▄▄▇▂▃▅▃▆▄▃▃▄▅▇▃▂▃▃▃▃▆▂▂▅▅
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▁▁▂▁▁▃▁▄▂▁▃▂▅▅▅▄▅▂▃▅▇▇▅▇▆█▅▄▆


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Linear           | 384   
1 | model       | GNNModel         | 17.0 K
2 | head        | Sequential       | 2.1 K 
3 | loss_module | CrossEntropyLoss | 0     
4 | metricf1    | BinaryF1Score    | 0     
5 | metricauc   | BinaryAUROC      | 0     
6 | pool        | GlobalAttention  | 65    
-------------------------------------------------
19.6 K    Trainable params
0         Non-trainable params
19.6 K    Total params
0.078     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=30` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc_epoch,▁▁▄▄▄▂▅▅▃▄▄▃▅▆▆▆▆▇▆▆█▆▆▆▇▆▇▇▇█
train_acc_step,▄▅▇▂▄▅▇▁▅▅█▅▁▆▇▇▄▇▅▁▄▅▅▄▅▇▁▂▅▇▅▄█▂▅▄▄▇▅▁
train_loss_epoch,█▇▆▅▄▇▃▄▅▆▄▅▅▃▃▃▃▃▂▂▁▂▂▂▂▃▂▂▃▂
train_loss_step,▄▃▃▄▄▃▃▅▃▁▃▄▄▄▁▃▃▂▃▃▅▄█▄▂▂▄▄▃▁▃▃▂▅▂▃▃▂▂▄
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▆▁▂▁▁▂▂▂▁▃▂▆▂▄▃▅▅▃▄▆▅▆▅▇▇▇█▇▆▇


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Linear           | 384   
1 | model       | GNNModel         | 17.0 K
2 | head        | Sequential       | 2.1 K 
3 | loss_module | CrossEntropyLoss | 0     
4 | metricf1    | BinaryF1Score    | 0     
5 | metricauc   | BinaryAUROC      | 0     
6 | pool        | Attention_module | 4.2 K 
-------------------------------------------------
23.7 K    Trainable params
0         Non-trainable params
23.7 K    Total params
0.095     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=30` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,▁
test_auc,▁
test_f1,▁
train_acc_epoch,▃▃▄▅▇▁▅▆▄▃▇▇▆▇▆▇▇▇▅▆▆▅▆▇▇▃█▇▅█
train_acc_step,▂▅▇▁▅▅▅▁▅▇▁█▇▁▅▅▇█▄▅▄▇▅▅▅▇▁▅▇▇▅▂▅▁▇▅▅▇▇▁
train_loss_epoch,█▇▅▅▄▇▃▄▄▆▅▃▅▄▃▂▁▂▆▄▁▃▄▃▃▄▂▂▄▃
train_loss_step,▃▁▂▄▃▂▃▄▃▂▅▂▂▃▁▂▁▁▃▂▃▂█▅▃▁▃▂▂▁▃▃▁▅▁▅▄▂▁▃
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▂▁▄▆▂▄▃▆█▆▄▆▇██▆▆▇▇▆▅▇▆▅▆█▇▇▇▆
